In [2]:
import pandas as pd
import requests
import plotly.express as px
import plotly.graph_objects as go

In [3]:
def geopos_from_zipcode(country_code, zip_code):
  API_KEY = "API_KEY"
  URL = f"https://api.openweathermap.org/geo/1.0/zip?zip={zip_code},{country_code}&appid={API_KEY}"
  response =  requests.get(URL)
  json = response.json()
  return json["lat"], json["lon"]

In [4]:
# Import Meteostat library and dependencies
from datetime import datetime
import plotly.express as px
from meteostat import Point, Hourly

# Set time period
start = datetime(2018, 1, 1)
end = datetime(2022, 12, 31)

# Create Point for Vancouver, BC
location = Point(*geopos_from_zipcode("DE", "71665"), 70)

# Get daily data for 2018
data = Hourly(location, start, end)
data = data.fetch()

In [5]:
px.line(data, y='temp')

In [36]:
ccodes = pd.read_csv("countries.csv", sep=" \t")

/tmp/ipykernel_72439/1301589277.py:1: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



In [39]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

app = Dash(__name__)

app.layout = html.Div([
  html.Label("Country Code"),
  dcc.Dropdown(id="country-dropdown", 
               options=ccodes["A2"]
               #options=[{"label": row["Country"], "value": row["A2"]} for _, row in ccodes.iterrows()]
               ),
  html.Label("Zip Code"),
  dcc.Input(id="zip-input", type="number", placeholder="Enter a Zip Code"),

  dcc.Graph(id="weather-graph")
])

@app.callback(
  Output("weather-graph", "figure"),
  Input("country-dropdown", "value"),
  Input("zip-input", "value")
)
def update_graph(country_code, zip_code):
  if country_code is None or zip_code is None:
    return {}
  
  # Set time period
  start = datetime(2021, 1, 1)
  end = datetime(2022, 12, 31)

  location = Point(*geopos_from_zipcode(country_code, zip_code), 500)
  data = Hourly(location, start, end)
  data = data.fetch()
  return px.line(data, y='temp')


In [42]:
app.run_server()

In [102]:
intensity_df

,Technologie Emissionen [g CO2eq/kWh],Min,Med,Max
0,Kohle,740,820,910
1,Biomasse kombiniert mit Kohle,620,740,890
2,Erdgas,410,490,650
3,Biomasse allein,130,230,420
4,Solarmodul in großem Maßstab,18,48,180
5,Solarmodul auf Dächern,26,41,60
6,Geothermische Energie,6,38,79
7,Solarenergie konzentriert 8,",8",27,63
8,Wasserkraft,1,24,2200
9,Windturbine auf See,8,12,35


In [112]:
intensity_df = pd.read_csv("data/co2intensity/co2intensities.csv", sep=";")

power_df = pd.read_csv("data/co2intensity/Actual_generation_201801010000_202301012359_Hour.csv", sep=";", thousands=",", decimal=".")
hour_deltas = pd.to_datetime(power_df["Start"]).dt.hour.apply(lambda x: pd.Timedelta(hours=x))
power_df["Date"] = pd.to_datetime(power_df["Date"]) + hour_deltas
power_df = power_df.drop(columns=["Start", "End"])
power_df.head()

/tmp/ipykernel_72439/3224734180.py:4: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



,Date,Biomass [MWh] Calculated resolutions,Hydropower [MWh] Calculated resolutions,Wind offshore [MWh] Calculated resolutions,Wind onshore [MWh] Calculated resolutions,Photovoltaics [MWh] Calculated resolutions,Other renewable [MWh] Calculated resolutions,Nuclear [MWh] Calculated resolutions,Lignite [MWh] Calculated resolutions,Hard coal [MWh] Calculated resolutions,Fossil gas [MWh] Calculated resolutions,Hydro pumped storage [MWh] Calculated resolutions,Other conventional [MWh] Calculated resolutions
0,2018-01-01 00:00:00,5032.75,5348.00,2914.25,29522.75,0.0,169.00,5853.75,6723.75,1862.50,2489.25,311.00,1888.00
1,2018-01-01 01:00:00,4998.00,5041.25,2997.75,30574.75,0.0,168.00,4754.00,6810.00,1760.00,2481.50,628.50,1924.50
2,2018-01-01 02:00:00,4992.75,4968.25,3093.25,31204.25,0.0,167.50,4591.00,6765.75,1832.75,2471.75,402.25,1854.50
3,2018-01-01 03:00:00,4992.50,4846.00,3134.50,31814.50,0.0,166.75,4888.75,6728.00,1828.50,2489.00,50.00,1812.25
4,2018-01-01 04:00:00,5006.75,4834.50,3209.50,31568.00,0.0,166.00,4696.50,6709.75,1826.50,2489.00,43.00,1858.25


In [100]:
px.line(power_df, x="Date", y=["Photovoltaics [MWh] Calculated resolutions", "Nuclear [MWh] Calculated resolutions", "Fossil gas [MWh] Calculated resolutions"])

/home/missinguser/CSE/fa2023/venv/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [119]:
for energy_type in power_df.columns:
  if (intensity_df["Emissions [g CO2eq/kWh]"] == energy_type).any():  
    print(energy_type)
    intensity_name = energy_type.replace("[MWh] Calculated resolutions", "[g CO2eq/kWh]")
    power_df[intensity_name] = power_df[energy_type] * intensity_df[intensity_df["Emissions [g CO2eq/kWh]"] == energy_type].at[0, "Med"]

Biomass [MWh] Calculated resolutions


KeyError: 'Med'

In [107]:
power_df

,Date,Biomass [MWh] Calculated resolutions,Hydropower [MWh] Calculated resolutions,Wind offshore [MWh] Calculated resolutions,Wind onshore [MWh] Calculated resolutions,Photovoltaics [MWh] Calculated resolutions,Other renewable [MWh] Calculated resolutions,Nuclear [MWh] Calculated resolutions,Lignite [MWh] Calculated resolutions,Hard coal [MWh] Calculated resolutions,Fossil gas [MWh] Calculated resolutions,Hydro pumped storage [MWh] Calculated resolutions,Other conventional [MWh] Calculated resolutions
0,2018-01-01 00:00:00,5032.75,5348.00,2914.25,29522.75,0.0,169.00,5853.75,6723.75,1862.50,2489.25,311.00,1888.00
1,2018-01-01 01:00:00,4998.00,5041.25,2997.75,30574.75,0.0,168.00,4754.00,6810.00,1760.00,2481.50,628.50,1924.50
2,2018-01-01 02:00:00,4992.75,4968.25,3093.25,31204.25,0.0,167.50,4591.00,6765.75,1832.75,2471.75,402.25,1854.50
3,2018-01-01 03:00:00,4992.50,4846.00,3134.50,31814.50,0.0,166.75,4888.75,6728.00,1828.50,2489.00,50.00,1812.25
4,2018-01-01 04:00:00,5006.75,4834.50,3209.50,31568.00,0.0,166.00,4696.50,6709.75,1826.50,2489.00,43.00,1858.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6570,2018-10-01 19:00:00,4844.50,4661.25,4415.75,14847.50,7.0,125.00,7862.75,15885.75,11753.50,7073.75,5957.75,2002.25
6571,2018-10-01 20:00:00,4836.75,4571.50,4204.75,15723.50,0.0,125.00,7825.25,15865.50,11327.00,6543.25,3251.50,1994.50
6572,2018-10-01 21:00:00,4826.25,4089.50,4047.25,15724.75,0.0,125.00,7757.00,15848.50,9838.00,6097.75,1061.00,2004.25
6573,2018-10-01 22:00:00,4815.00,4043.25,3532.75,15362.75,0.0,125.00,7715.00,15708.75,8223.00,5285.50,960.75,2011.75
